### 4강 기본모델 구축 + 저장
---

In [2]:
# 이미지 처리에 성과가 좋은 CNN 모델을 구현해보자

In [3]:
# 라이브러리 불러오기
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

---  
## Load data set

In [4]:
# 데이터와 어떻게 불러올지 설정

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='D:/chchdata/dataset', train=True, download=True, transform = transform)
trainloader = DataLoader(trainset, batch_size=8, shuffle = True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='D:/chchdata/dataset', train = False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=8, shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat','deer','dog','frog','horse','ship','truck')

Files already downloaded and verified
Files already downloaded and verified


---
## Build a model

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):  # nn.modules 를 상속받음
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)  # 사이즈를 변형하는 레이어(1자로 펴는 작업)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [6]:
# 모델 확인
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


---
## Implement the model with training set

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # net.parameters() : 이 부분이 모델의 파라미터를 업데이트한다는 뜻

In [8]:
# 실직적인 학습 부분

for epoch in range(1) :                                                 # epoch 횟수 설정

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):                           # 트레인로더에서 트레인 데이터 불러오기
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data                                           # 데이터 안에는 이미지, 라벨이 있음

        # zero the parameter gradients
        optimizer.zero_grad()                                           # 옵티마이저 초기화 작업

        # forward + backward + optimizer
        outputs = net(inputs)                                           # 이미지를 net 모델에 넣기
        loss = criterion(outputs, labels)                               # outputs과 labels을 비교해서 loss 계산
        loss.backward()                                                 # loss를 기준으로 gradient를 계산해서 backward를 진행
        optimizer.step()                                                # 이렇게 한 번 돌면 배치크기 만큼 돌리면서 loss를 누적하게 됨

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1000 :                                           # 설정한 값 마다 loss 출력
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    
print('Finished Training')

C:\Users\lemon\anaconda3\envs\chch\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,  1001] loss: 1.129
[1,  3001] loss: 1.888
[1,  5001] loss: 1.626
Finished Training


---  
## save the trained model

In [9]:
PATH = 'D:/chchdata/pth/cifar_net.pth'  # 모델에서 나온 가중치를 저장할 경로 지정
torch.save(net.state_dict(), PATH)      # model이름.state_dict()

---
## load the pre-trained model

In [10]:
net = Net()
net.load_state_dict(torch.load(PATH))
# 잘 불러와짐

<All keys matched successfully>

---
## test에 적용  
output은 미니배치의 결과가 산출되기 때문에 for문을 통해서 test 전체의 예측값을 구해야 한다.

In [12]:
correct = 0
total = 0
with torch.no_grad():   # 업데이트를 안 하는 것으로 설정
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)   # == argmax / 10개의 라벨이 1개로 바뀐다
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('1000개의 test 이미지에 대한 모델 가중치 정확도: %d %%'%(100 * correct / total))


1000개의 test 이미지에 대한 모델 가중치 정확도: 45 %


gpu로 연산하게 하는 방법은 포함되지 않았음